# План

Шаг 1. Открыть файл с данными и изучить общую информацию

Шаг 2. Предобработка данных

Шаг 3. Ответить на вопросы.

Шаг 4. Вывод

In [1]:
# Импорт нужных библиотек
import pandas as pd
from pymystem3 import Mystem 

### Шаг 1. Открою файл с данными и изучу общую информацию. 

In [2]:
# Читаю файл с данными и называю его data
data = pd.read_csv('/datasets/data.csv') 

# Вывожу общую информацию 
data.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [4]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
# Смотрю пропущенные значения
data.isnull().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
# Смотрю уникальные значения столбцов
for column in ['children','dob_years','gender']:
    display(data[column].unique() )

array([ 1,  0,  3,  2, -1,  4, 20,  5])

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

array(['F', 'M', 'XNA'], dtype=object)

### Вывод


Есть пропуски в данных.
Есть некорректные значения детей: **-1 и 20** (Например 606 его возраст 21 и у него 20 детей).
Со стажем тоже проблемы большая часть записана с **-** но есть и те которые без минуса и там стажа **1000 лет** .
Есть некорректный возраст **0** .
Есть не правильная запись образования( **разный регистр** ).
В столбце пол есть пол: **XNA** (1 клиент).
В столбце цель кредита надо сделать группировку т.к слишком много схожих целей.



### Шаг 2. Предобработка данных

### Обработка пропусков

In [7]:
def func_for_med(column , med):
    """
    Меняет значения NaN в колонке total_income
    column - тут пишем  должность, чтобы менять значение не во всём столбце, а более выборочно
    med- это на что меняем. Далее в проекте я меняю на медиану
    """
    data.loc[data['income_type'] == column, 'total_income'] = data.loc[data['income_type'] == column, 'total_income'].fillna(med)


def med(query):
    """
    Возвращает медиану 
    """
    return data[data['income_type'] == query]['total_income'].median()
    

for query in ['сотрудник','пенсионер','компаньон','госслужащий']:
    func_for_med(query,med(query))

# Т.к предпринимателя всего 2 и у одного не указан доход я взял медиану от всех   
func_for_med('предприниматель' , data['total_income'].median())


data['days_employed'] = data['days_employed'].fillna(0) # стаж поменял на 0 так как он в дальнейшем нам не нужен и 
#есть аномальные данные в 1000 лет стажа и с отрицательными значениями

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

**есть пояснение, какие пропущенные значения обнаружены.**
Были обнаружены значения `NaN` в столбце *total_income* и *days_employed*
Так же в *dob_years* есть значение *0* . В *days_employed* есть стаж *1000 лет*
***
**описаны возможные причины появления пропусков в данных.**
Возможно данные внесли не корректно и данные потерялись или их не было изначально. Клиент не указывал свой доход и стаж специально или это не главное для банка
***

Тут я использую функцию `func_for_med`
Были заполнены пропуски медианой по типу занятости(сотрудник, пенсионер, компаньон, госслужащий)
У нас есть всего два предпринимателя и у одного из них пропущено значение,его я заполнил медианной от всех.
Так же в стаже я заменил пустые значения на **0**

### Замена типа данных

In [8]:
def sub_zero(row):
    """
Меняет отрицательное число на положительное и преобразует в целочисленное.
    """
    if row < 0:
        return int(row * -1)
    return int(row)

for column in ['days_employed','total_income','children']:
    data[column] = data[column].apply(sub_zero)
    
data['education'] = data['education'].str.lower() # переводим всё в нижний регист

data.head(10) # проверил изменилось ли таблица и корректно ли

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

С помощью функции `sub_zero` в столбцах *days_employed* и *total_income* поменял значения на целочисленные и перевел отрицательные числа в положительные, так с помощью этой функции в *children* изменил кол-во детей с *-1* на *1*
В *education* перевел всё в нижний регистр
***

*В столбце дети есть данные у кого их 20,возможно это опечатка и их должно быть 2. Т.к их всего 76 (а это меньше 0.01 % всех данных) и они не будут влиять на наши данные я делаю гипотезу что это правда и у этих людей есть 20 детей (Свои, усыновлённые)*



### Обработка дубликатов

In [9]:
data = data.drop_duplicates().reset_index(drop = True)

data.duplicated().sum()

0

### Вывод

Удалил дубликаты с помощью `drop_duplicates` и сбросил индекс используя `reset_index(drop = True)`
***
**описаны возможные причины появления дубликатов в данных.**
*Тут я затрудняюсь ответить.*
Возможно была ошибка и данные были занесены два раза или файлов было много, но маленького объёма и перед объединением файла конец старого и начала нового были одинаковые данные

### Лемматизация

In [10]:
# из библиотеки pymystem3
m = Mystem()

def group_lemma(row): # всего я разделил на 5 групп 
    """
    Делает группы по лемме
    """
    if ('автомобиль' in row):
        return 'приобретение автомобиля'
    if ('свадьба' in row):
        return 'на свадьбу'
    if ('образование' in row):
        return 'образование'
    if ('коммерческий' in row) &  ('недвижимость' in row):
        return 'операции с коммерческой недвижимостью'    
    else:
        return 'операции с недвижимостью'    

    

# Использовал функцию для вывода лемм и перенес их в отдельный столбец    
data['purpose_lemma'] = data['purpose'].apply(m.lemmatize) 

# использовал свою функцию и разделил цели на 5 категорий 
data['purpose_group'] = data['purpose_lemma'].apply(group_lemma) 

data['purpose_group'].value_counts() # посчитал сколько людей хотят кредит и на что 

операции с недвижимостью                 9500
приобретение автомобиля                  4306
образование                              4013
на свадьбу                               2324
операции с коммерческой недвижимостью    1311
Name: purpose_group, dtype: int64

### Вывод

Использовав библиотеку `pymystem3` я сделал разделение по лемме.
Далее я использовал свою функцию `group_lemma` и сгруппировал их в 5 групп *приобретение автомобиля, на свадьбу, образование, операции с коммерческой недвижимостью и операции с недвижимостью*.
Из полученных данных мы видим что кредит *на операции с недвижимостью* берут больше чем на *приобретение автомобиля* и *образование* вместе взятых. Тут так же видно что люди берут кредит *на свадьбу* в два раза чаще чем на *операции с коммерческой недвижимостью*

### Категоризация данных

In [11]:
def group_income(row): # Эта функция разделяет доход на 3 группы 
    """
    Делает группы по доходу
    """
    if 100000 > row:
        return 'низкий'
    if 100000 <= row <= 200000:
        return 'средний'
    else:
        return 'высокий'    

data['group_total_income'] = data['total_income'].apply(group_income) # Сделал группировку по доходу

data.head(10) # Вывел  первые 10 строк информации для проверки корректности группировок

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,purpose_group,group_total_income
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",операции с недвижимостью,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",приобретение автомобиля,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",операции с недвижимостью,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",на свадьбу,средний
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",операции с недвижимостью,высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",операции с недвижимостью,высокий
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование,средний
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",на свадьбу,низкий
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",операции с недвижимостью,средний


### Вывод

***Категоризация данных***
В лемматизации я описал что сделал группировку по целям.
Здесь я использовал функцию `group_income` и сделал категорию по заработку *низкий*, *средний* и *высокий*. Далее эта группа мне пригодиться для ответа на вопрос.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [12]:
data_pivot = data.pivot_table(index =['children'], columns = [],values = ['debt'], aggfunc = 'count') 
# создаю сводную таблицу общего количества 

data_pivot['debtor'] = data.pivot_table(index =['children'], columns = [],values = ['debt'], aggfunc = 'sum')
# кол-во с задолжностью

data_pivot['share'] = (data_pivot['debtor'] / data_pivot['debt'] * 100) 
# % соотношение которое показывает влиет ли дети на возврат кредита
data_pivot

,debt,debtor,share
children,,,
0,14091,1063,7.543822
1,4855,445,9.165808
2,2052,194,9.454191
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000
20,76,8,10.526316


### Вывод

**Есть ли зависимость между наличием детей и возвратом кредита в срок?**
Нет зависимости от наличия детей и задолженности кредита. Т.к задолженность кредита не превышает 10% как у людей с детьми так и без них.Выборку с *20* детьми я не могу считать корректной и на неё не обращаю внимание.
Как видно в таблице люди без детей берут кредит чаще и *7.5%* имели задолженность по возврату кредитов. 
***
Можно предположить другие гипотезы что на невозврат могут влиять такие факторы как зарплата, семейное положение, тип занятости, возраст, цель кредита или совокупность всех факторов. А может быть наоборот на невозврат кредита влияет те факторы которые не учтены и не могут быть учтены.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?


In [13]:
data_family = data.pivot_table(index =['family_status'], columns = [],values = ['debt'], aggfunc = 'count',)

data_family['debtor'] = data.pivot_table(index =['family_status'], columns = [],values = ['debt'], aggfunc = 'sum')

data_family['share'] = (data_family['debtor'] / data_family['debt'] * 100) 

data_family

,debt,debtor,share
family_status,,,
Не женат / не замужем,2810,274,9.750890
в разводе,1195,85,7.112971
вдовец / вдова,959,63,6.569343
гражданский брак,4151,388,9.347145
женат / замужем,12339,931,7.545182


### Вывод

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**
Нет, данные так же не превышают 10% 


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [14]:
data_pivot_income = data.pivot_table(index =['group_total_income'], columns = [],values = ['debt'], aggfunc = ['count', 'sum'])

data_pivot_income['share'] = (data_pivot_income['sum'] /data_pivot_income['count']) * 100

data_pivot_income

,count,sum,share
,debt,debt,
group_total_income,,,
высокий,5066,358,7.066719
низкий,4463,354,7.931884
средний,11925,1029,8.628931


### Вывод

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**
Нет, данные так же не превышают 10%


- Как разные цели кредита влияют на его возврат в срок?

In [15]:
data_pivot_purpose = data.pivot_table(index =['purpose_group'], columns = [],values = ['debt'], aggfunc = ['count', 'sum'])

data_pivot_purpose['share'] = (data_pivot_purpose['sum'] / data_pivot_purpose['count']) * 100

data_pivot_purpose

,count,sum,share
,debt,debt,
purpose_group,,,
на свадьбу,2324,186,8.003442
образование,4013,370,9.220035
операции с коммерческой недвижимостью,1311,99,7.551487
операции с недвижимостью,9500,683,7.189474
приобретение автомобиля,4306,403,9.359034


### Вывод

**Как разные цели кредита влияют на его возврат в срок?**
Как видно из таблицы на *приобретение автомобиля* и *образование* отдают вовремя реже чем в сравнение с *операции с недвижимостью* но и берут реже

### Шаг 4. Общий вывод

Проанализировав полученные данные можно сказать что задолженность по возврату кредитов есть у каждого 10 человека. Следовательно, на задолженность по возврату кредита влияет те факторы которых я не проанализировал или которых нет в этих данных
***
*Ниже я вывел таблицу и посмотрел какой тип людей по *образованию* и *должности* имел задолженность по возврату кредитов и на какие цели.* 

In [16]:
data_pivotat = data.pivot_table(index =['income_type', 'education'], columns = ['purpose_group'],values = ['debt'], aggfunc = 'sum')

data_pivotat.fillna(0) # Заменил значения Nan на 0

debt              \
purpose_group                       на свадьбу образование   
income_type     education                                    
безработный     высшее                     0.0         0.0   
                среднее                    0.0         0.0   
в декрете       среднее                    0.0         0.0   
госслужащий     высшее                     1.0         3.0   
                начальное                  0.0         1.0   
                неоконченное высшее        0.0         0.0   
                среднее                    6.0        17.0   
                ученая степень             0.0         0.0   
компаньон       высшее                    18.0        18.0   
                начальное                  1.0         0.0   
                неоконченное высшее        2.0         2.0   
                среднее                   31.0        52.0   
пенсионер       высшее                     1.0         6.0   
                начальное                  1.0         1.0   
                неоконченное высшее        0.0         0.0   
                среднее                   23.0        41.0   
                ученая степень             0.0         0.0   
предприниматель высшее                     0.0         0.0   
сотрудник       высшее                     7.0        28.0   
                начальное                  2.0         3.0   
                неоконченное высшее        4.0        10.0   
                среднее                   89.0       188.0   
                ученая степень             0.0         0.0   
студент         высшее                     0.0         0.0   

                                                                           \
purpose_group                       операции с коммерческой недвижимостью   
income_type     education                                                   
безработный     высшее                                                0.0   
                среднее                                               0.0   
в декрете       среднее                                               0.0   
госслужащий     высшее                                                1.0   
                начальное                                             0.0   
                неоконченное высшее                                   0.0   
                среднее                                               3.0   
                ученая степень                                        0.0   
компаньон       высшее                                                3.0   
                начальное                                             0.0   
                неоконченное высшее                                   1.0   
                среднее                                              13.0   
пенсионер       высшее                                                3.0   
                начальное                                             0.0   
                неоконченное высшее                                   0.0   
                среднее                                              10.0   
                ученая степень                                        0.0   
предприниматель высшее                                                0.0   
сотрудник       высшее                                                9.0   
                начальное                                             1.0   
                неоконченное высшее                                   1.0   
                среднее                                              54.0   
                ученая степень                                        0.0   
студент         высшее                                                0.0   

                                                              \
purpose_group                       операции с недвижимостью   
income_type     education                                      
безработный     высшее                                   0.0   
                среднее        

Из этой таблицы следует  что сотрудники со средним образованием чаще всего имею задолженности по возврату кредитов